---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [19]:
import pandas as pd
import numpy as np

def get_list_of_university_towns():
    unitown = pd.read_table('university_towns.txt',header = None, names = ['state&region'])
    state = ''
    def get_state(target):
        if target.endswith('[edit]'):
            global state
            state = target[:6]
            return target[:6]
        else:
            return state
    unitown['State']= unitown['state&region'].map(get_state)#根据条件赋值
    unitown =unitown[unitown['state&region'].str.endswith('[edit]')==False]#删除多余的state行
    unitown['RegionName'] = unitown['state&region'].map(lambda x: x.split('(')[0])
    unitown = unitown.reset_index()
    unitown = unitown[['State','RegionName']]
    
    '''~逐位取反[&：按位与操作，只有 1 &1 为1，其他情况为0。可用于进位运算。
            |：按位或操作，只有 0|0为0，其他情况为1。
            ~：逐位取反。
            ^：异或，相同为0，相异为1。可用于加操作（不包括进位项）。
            <<：左移操作，2的幂相关
            >>：右移操作，2的幂相关]'''
    
    
    
    
    '''要求：Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return unitown
get_list_of_university_towns()

,State,RegionName
0,Alabam,Auburn
1,Alabam,Florence
2,Alabam,Jacksonville
3,Alabam,Livingston
4,Alabam,Montevallo
...,...,...
512,Wiscon,River Falls
513,Wiscon,Stevens Point
514,Wiscon,Waukesha
515,Wiscon,Whitewater


In [20]:
gdp = (pd.read_excel('gdplev.xls',skiprows = 7,usecols = [4,6],names = ['year&quarter', 'GDP'])
                  .iloc[211:]
                  .reset_index(drop=True))  #reset_index(drop=True)删除原index 
gdp['GDP'].astype(np.float64)
    #经济衰退：从gdp连续降低两个季度开始，到gdp连续增加两个季度结束。
    #增加阶段 减少阶段 
    #if减少阶段多于2个季度,则存在start;if减少阶段结束后增加阶段多于2个季度,则存在end
delta = [np.nan] 
for i in range(1,len(gdp)):
    if gdp.iloc[i]['GDP']>gdp.iloc[i-1]['GDP']:
        delta.append(1)
    else:
        delta.append(0)

for i in range(3,len(delta)):
    if delta[i-1]==1 and delta[i]==1 and delta[i-2]==0 and delta[i-3]==0:
        end_index = i
        end = gdp['year&quarter'].iloc[i]
        for j in range(4,i+1):
            if delta[i-j]==1:
                start_index = i-j+1
                start = gdp['year&quarter'].iloc[i-j+1]
                break
                #先确定start，再确定end，则要在start_index i上+4 +5 ...+j
'''
gdplev=pd.read_excel('gdplev.xls', skiprows=218, usecols=[4,6], names=['Quarter','GDP'])
growth=[np.nan]
for i in range(1,len(gdplev)):
    if gdplev['GDP'][i]>gdplev['GDP'][i-1]:
        growth.append(1)
    else:
        growth.append(0)
for i in range(3, len(growth)):
    if growth[i-1]==1 and growth[i]==1:
        if growth[i-2]==0 and growth[i-3]==0:
            end=gdplev['Quarter'][i]
            end_index=i
            for j in range(4,i+1):
                if growth[i-j]==1:
                    start=gdplev['Quarter'][i-j+1]
                    start_index=i-j+1
                    break
'''
def get_recession_start():
   
        
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return start
get_recession_start()

'2008q3'

In [21]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return end
get_recession_end()

'2009q4'

In [22]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    bottom_index = gdp['GDP'].iloc[start_index:end_index].idxmin()
    bottom = gdp['year&quarter'].iloc[bottom_index]
    return bottom
get_recession_bottom()

'2009q2'

In [23]:
def convert_housing_data_to_quarters():
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    ihouse = housing.loc[:,'2000-01':'2016-08']
    ihouse.columns = pd.to_datetime(ihouse.columns).to_period(freq='M')
    ihouse = ihouse.groupby(ihouse.columns.asfreq('Q'),axis=1).mean()
    ihouse.columns = ihouse.columns.map(lambda x: str(x).replace('Q','q'))
    housing['State']=housing['State'].replace(states)
    ihouse = ihouse.set_index([housing['State'],housing['RegionName']])
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return ihouse
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"